## Imports

In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from bitcoin_deep_learning.call_api import ApiCall
from bitcoin_deep_learning.model import RnnDlModel, LinearRegressionBaselineModel, DummyModel, RandomForestReg, RnnDlModel_test 
from bitcoin_deep_learning.trainer import train, read_result
from bitcoin_deep_learning.cross_val import get_cross_XY
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import random
import joblib
%load_ext autoreload
%autoreload 2

## Getting data

In [2]:
train_df = ApiCall().get_raw_glassnode_data()
#read_local(data='train')#.drop(columns = '[%]_Bitcoin_growth_rate_on_Horizon=7')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:35<00:00,  1.30s/it]


In [3]:
train_df.head()

,date,[AVG]_[NH]_mean_hash_rate,[+]_[NH]_Days_Till_Halving,[//]_[AV]_Stock-to-Flow_Ratio,[+]_[NH]_Circulating_Supply,[+]_[NH]_Issuance,[+]_[NH]_Number_of_Active_Addresses,[+]_[NH]_Mean_Block_Interval,[+]_[NH]_Number_of_Transactions,[+]_[NH]_Number_of_Addresses_with_a_Non-Zero_Balance,...,[$]_[BSB]_Exchange_Net_Position_Change_-_All_Exchanges,[//]_[BSB]_Realized_Profit/Loss_Ratio,[$]_[BSB]_Net_Unrealized_Profit/Loss_(NUPL),[$]_[BSB]_Realized_Price,[%]_[BSB]_Price_Drawdown_from_ATH,[//]_[AV]_Market_Value_to_Realized_Value_Ratio_(MVRV),[//]_[AV]_Puell_Multiple,[//]_[AV]_Realized_Profits-to-Value_(RPV)_Ratio,[+]_[T]_Bitcoin_Price,[%]_Bitcoin_growth_rate_on_Horizon=7
0,2011-09-16,1.036434e+13,439,1.418819,7282650.0,5950.0,14298,508,6090,494042,...,74.789093,0.130762,-112113.115163,6.124551,-0.849159,0.785527,0.683785,0.000378,4.811000,NaN
1,2011-09-17,1.161376e+13,438,1.429329,7289350.0,6700.0,12877,455,5320,494415,...,69.740065,0.173719,-42045.963095,6.118551,-0.850611,0.778730,0.766186,0.000198,4.764700,NaN
2,2011-09-18,1.363396e+13,437,1.441211,7297250.0,7900.0,14074,349,5828,494962,...,30.148647,0.387620,-26346.007227,6.113585,-0.835764,0.856819,0.913495,0.000374,5.238233,NaN
3,2011-09-19,1.113934e+13,436,1.452215,7303600.0,6350.0,15826,454,6526,495218,...,38.196675,0.293976,-54997.251969,6.105355,-0.828724,0.894751,0.782018,0.000514,5.462771,NaN
4,2011-09-20,1.214578e+13,435,1.462380,7310600.0,7000.0,21668,422,8005,499994,...,36.377675,0.853582,-11317.089254,6.104715,-0.808281,1.001652,0.981353,0.001478,6.114797,NaN


In [ ]:
train_df.drop(columns = '')

In [74]:
X_train_list, Y_train_list, X_test_list,Y_test_list = get_cross_XY(d)

In [75]:
np.array(X_train_list).shape

(32, 263, 90, 32)

In [61]:
np.array(Y_test_list).shape

(32, 90)

In [62]:
train_df.head(3)

,date,[AVG]_[NH]_mean_hash_rate,[+]_[NH]_Days_Till_Halving,[//]_[AV]_Stock-to-Flow_Ratio,[+]_[NH]_Circulating_Supply,[+]_[NH]_Issuance,[+]_[NH]_Number_of_Active_Addresses,[+]_[NH]_Mean_Block_Interval,[+]_[NH]_Number_of_Transactions,[+]_[NH]_Number_of_Addresses_with_a_Non-Zero_Balance,...,[//]_[AV]_Realized_Profits-to-Value_(RPV)_Ratio,fear_greed_value,[+]_[NH]_Number_of_Addresses_with_Balance_0.01 - 0.1,[+]_[NH]_Number_of_Addresses_with_Balance_0.1 - 1,[+]_[NH]_Number_of_Addresses_with_Balance_1 - 10,[+]_[NH]_Number_of_Addresses_with_Balance_10 - 100,[+]_[NH]_Number_of_Addresses_with_Balance_100 - 1k,[+]_[NH]_Number_of_Addresses_with_Balance_1k - 10k,[+]_[T]_Bitcoin_Price,[%]_Bitcoin_growth_rate_on_Horizon=7
0,2018-02-01,2.054109e+19,830,5642.903220,16839700.0,2000.0,842398,377,257504,27026398,...,0.003691,30,4226306,1722331,544769,132314,15779,1529,9014.026207,-0.235616
1,2018-02-02,2.170824e+19,829,5641.695528,16841800.0,2100.0,854253,385,235750,26955617,...,0.002557,15,4198274,1704275,543489,132213,15815,1521,8799.121911,-0.260491
2,2018-02-03,2.073919e+19,828,5641.015128,16843775.0,1975.0,714655,384,194733,26886977,...,0.001970,40,4173275,1692828,542447,132021,15812,1523,9208.459919,-0.230759


## Ploting models

### Plotting prediction per folds

In [163]:
def plot_predictions_models_all_folds(X_train_list, Y_train_list, X_test_list, Y_test_list, model):
    fig, axs = plt.subplots(nrows = len(X_train_list), ncols = 4, figsize = (25,200))
    scores = []
    for k in range(len(X_train_list)): #range(5):#
        X_train, y_train, X_test, y_test = X_train_list[k], Y_train_list[k], X_test_list[k], Y_test_list[k]
        #intuition too much regularization on the lasso
        model.set_model()
        y_pred = model.run(X_test, X_train, y_train)
        score = mean_absolute_error(y_test, y_pred)
        scores.append(score)
        axs[k, 0].plot(y_test, label = 'test')
        axs[k, 0].plot(y_pred, color = 'r', label = 'pred')
        axs[k, 0].set_title('Fitting of y_test, y_pred')
        axs[k, 0].set_ylabel('Price growth rate 7 days')
        axs[k, 0].set_xlabel('Fold sequence length in days')
        axs[k, 0].legend(loc='upper right')
        axs[k, 1].plot(y_train, label = 'train', color = 'g')
        axs[k, 1].set_title('Evolution of the y_train')
        axs[k, 1].set_ylabel('Price growth rate 7 days')
        axs[k, 1].set_xlabel('Fold sequence length in days')
        axs[k, 1].legend(loc='upper right');
        axs[k, 2].plot(model.history.history['mae'], color = 'y', label = 'mae')
        axs[k, 2].plot(model.history.history['val_mae'], color = 'b', label = 'val_mae')
        axs[k, 2].set_title('model mae')
        axs[k, 2].set_ylabel('mae')
        axs[k, 2].set_xlabel('epoch')
        axs[k, 2].legend(['train', 'test'], loc='upper right')
        axs[k, 3].plot(model.history.history['loss'], color = 'y', label = 'mae')
        axs[k, 3].plot(model.history.history['val_loss'], color = 'b', label = 'val_mae')
        axs[k, 3].set_title('model train vs validation loss')
        axs[k, 3].set_ylabel('loss')
        axs[k, 3].set_xlabel('epoch')
        axs[k, 3].legend(['train', 'validation'], loc='upper right')
        if k == 1:
            print(model.model.summary())
        print(f'############### next fold => number {k} ########## ')
    print("Mean absolute error - mean") 
    print(np.array(scores).mean())
    print("Mean absolute error - std") 
    print(np.array(scores).std())
    

### RNN Tuning

#### Tuning RNN

In [ ]:
l1_rnn = 0.01653480436240602
l2_rnn = 0.0792386474

plot_predictions_models_all_folds(X_train_list, Y_train_list, X_test_list, Y_test_list, RnnDlModel_test(L1 = l1_rnn, L2 = l2_rnn, epochs = 250, patience = 15))



Epoch 1/250
4/4 [==============================] - 3s 196ms/step - loss: 0.1381 - mae: 0.1288 - val_loss: 0.0618 - val_mae: 0.0768
Epoch 2/250
4/4 [==============================] - 0s 68ms/step - loss: 0.0573 - mae: 0.0767 - val_loss: 0.0449 - val_mae: 0.0751
Epoch 3/250
4/4 [==============================] - 0s 62ms/step - loss: 0.0549 - mae: 0.0732 - val_loss: 0.0467 - val_mae: 0.0771
Epoch 4/250
4/4 [==============================] - 0s 66ms/step - loss: 0.0482 - mae: 0.0753 - val_loss: 0.0577 - val_mae: 0.0881
Epoch 5/250
4/4 [==============================] - 0s 65ms/step - loss: 0.0504 - mae: 0.0779 - val_loss: 0.0459 - val_mae: 0.0774
Epoch 6/250
4/4 [==============================] - 0s 61ms/step - loss: 0.0506 - mae: 0.0771 - val_loss: 0.0434 - val_mae: 0.0738
Epoch 7/250
4/4 [==============================] - 0s 62ms/step - loss: 0.0485 - mae: 0.0737 - val_loss: 0.0433 - val_mae: 0.0736
Epoch 8/250
4/4 [==============================] - 0s 62ms/step - loss: 0.0492 - mae: 0.0

4/4 [==============================] - 0s 62ms/step - loss: 0.0421 - mae: 0.0651 - val_loss: 0.0130 - val_mae: 0.0424
Epoch 12/250
4/4 [==============================] - 0s 63ms/step - loss: 0.0500 - mae: 0.0682 - val_loss: 0.0124 - val_mae: 0.0408
Epoch 13/250
4/4 [==============================] - 0s 62ms/step - loss: 0.0409 - mae: 0.0640 - val_loss: 0.0132 - val_mae: 0.0430
Epoch 14/250
4/4 [==============================] - 0s 63ms/step - loss: 0.0407 - mae: 0.0634 - val_loss: 0.1099 - val_mae: 0.0936
Epoch 15/250
4/4 [==============================] - 0s 63ms/step - loss: 0.0411 - mae: 0.0661 - val_loss: 0.0658 - val_mae: 0.0620
Epoch 16/250
4/4 [==============================] - 0s 62ms/step - loss: 0.0348 - mae: 0.0634 - val_loss: 0.0126 - val_mae: 0.0414
Epoch 17/250
4/4 [==============================] - 0s 64ms/step - loss: 0.0439 - mae: 0.0637 - val_loss: 0.0324 - val_mae: 0.0554
Epoch 18/250
4/4 [==============================] - 0s 62ms/step - loss: 0.0350 - mae: 0.0626 - 

Epoch 27/250
4/4 [==============================] - 0s 64ms/step - loss: 0.0302 - mae: 0.0574 - val_loss: 0.0363 - val_mae: 0.0948
Epoch 28/250
4/4 [==============================] - 0s 63ms/step - loss: 0.0281 - mae: 0.0547 - val_loss: 0.0393 - val_mae: 0.0991
Epoch 29/250
4/4 [==============================] - 0s 63ms/step - loss: 0.0281 - mae: 0.0567 - val_loss: 0.0372 - val_mae: 0.0963
Epoch 30/250
4/4 [==============================] - 0s 63ms/step - loss: 0.0267 - mae: 0.0564 - val_loss: 0.0314 - val_mae: 0.0869
Epoch 31/250
4/4 [==============================] - 0s 64ms/step - loss: 0.0330 - mae: 0.0640 - val_loss: 0.0379 - val_mae: 0.0972
############### next fold => number 5 ########## 
Epoch 1/250
4/4 [==============================] - 4s 195ms/step - loss: 0.0887 - mae: 0.0962 - val_loss: 0.1465 - val_mae: 0.1379
Epoch 2/250
4/4 [==============================] - 0s 66ms/step - loss: 0.0771 - mae: 0.0896 - val_loss: 0.0923 - val_mae: 0.1029
Epoch 3/250
4/4 [=================

Epoch 38/250
4/4 [==============================] - 0s 64ms/step - loss: 0.0314 - mae: 0.0723 - val_loss: 0.0267 - val_mae: 0.0561
Epoch 39/250
4/4 [==============================] - 0s 63ms/step - loss: 0.0385 - mae: 0.0774 - val_loss: 0.0317 - val_mae: 0.0562
Epoch 40/250
4/4 [==============================] - 0s 63ms/step - loss: 0.0323 - mae: 0.0715 - val_loss: 0.0868 - val_mae: 0.1027
Epoch 41/250
4/4 [==============================] - 0s 62ms/step - loss: 0.0573 - mae: 0.0835 - val_loss: 0.0327 - val_mae: 0.0538
Epoch 42/250
4/4 [==============================] - 0s 64ms/step - loss: 0.0313 - mae: 0.0724 - val_loss: 0.0398 - val_mae: 0.0635
Epoch 43/250
4/4 [==============================] - 0s 64ms/step - loss: 0.0398 - mae: 0.0777 - val_loss: 0.0143 - val_mae: 0.0517
Epoch 44/250
4/4 [==============================] - 0s 63ms/step - loss: 0.0309 - mae: 0.0715 - val_loss: 0.0258 - val_mae: 0.0561
Epoch 45/250
4/4 [==============================] - 0s 63ms/step - loss: 0.0310 - m

Epoch 20/250
4/4 [==============================] - 0s 63ms/step - loss: 0.0440 - mae: 0.0721 - val_loss: 0.0121 - val_mae: 0.0490
Epoch 21/250
4/4 [==============================] - 0s 65ms/step - loss: 0.0517 - mae: 0.0742 - val_loss: 0.0317 - val_mae: 0.0561
Epoch 22/250
4/4 [==============================] - 0s 63ms/step - loss: 0.0467 - mae: 0.0713 - val_loss: 0.0391 - val_mae: 0.0617
Epoch 23/250
4/4 [==============================] - 0s 64ms/step - loss: 0.0613 - mae: 0.0778 - val_loss: 0.0146 - val_mae: 0.0526
Epoch 24/250
4/4 [==============================] - 0s 64ms/step - loss: 0.0436 - mae: 0.0711 - val_loss: 0.0428 - val_mae: 0.0647
Epoch 25/250
4/4 [==============================] - 0s 65ms/step - loss: 0.0442 - mae: 0.0715 - val_loss: 0.0329 - val_mae: 0.0569
Epoch 26/250
4/4 [==============================] - 0s 63ms/step - loss: 0.0441 - mae: 0.0723 - val_loss: 0.0139 - val_mae: 0.0516
Epoch 27/250
4/4 [==============================] - 0s 63ms/step - loss: 0.0367 - m

Epoch 12/250
4/4 [==============================] - 0s 71ms/step - loss: 0.1065 - mae: 0.0956 - val_loss: 0.1427 - val_mae: 0.1192
Epoch 13/250
4/4 [==============================] - 0s 64ms/step - loss: 0.0965 - mae: 0.0842 - val_loss: 0.1120 - val_mae: 0.1021
Epoch 14/250
4/4 [==============================] - 0s 65ms/step - loss: 0.0926 - mae: 0.0833 - val_loss: 0.0463 - val_mae: 0.0680
Epoch 15/250
4/4 [==============================] - 0s 64ms/step - loss: 0.0969 - mae: 0.0857 - val_loss: 0.0881 - val_mae: 0.0882
Epoch 16/250
4/4 [==============================] - 0s 63ms/step - loss: 0.0882 - mae: 0.0813 - val_loss: 0.0176 - val_mae: 0.0594
############### next fold => number 16 ########## 
Epoch 1/250
4/4 [==============================] - 3s 189ms/step - loss: 0.5308 - mae: 0.2576 - val_loss: 0.0418 - val_mae: 0.0645
Epoch 2/250
4/4 [==============================] - 0s 67ms/step - loss: 0.1138 - mae: 0.0810 - val_loss: 0.0322 - val_mae: 0.0627
Epoch 3/250
4/4 [================

4/4 [==============================] - 0s 64ms/step - loss: 0.0921 - mae: 0.0801 - val_loss: 0.0310 - val_mae: 0.0558
Epoch 38/250
4/4 [==============================] - 0s 63ms/step - loss: 0.0777 - mae: 0.0708 - val_loss: 0.0167 - val_mae: 0.0519
Epoch 39/250
4/4 [==============================] - 0s 63ms/step - loss: 0.0596 - mae: 0.0674 - val_loss: 0.0176 - val_mae: 0.0517
Epoch 40/250
4/4 [==============================] - 0s 60ms/step - loss: 0.0737 - mae: 0.0765 - val_loss: 0.0256 - val_mae: 0.0545
Epoch 41/250
4/4 [==============================] - 0s 63ms/step - loss: 0.0846 - mae: 0.0728 - val_loss: 0.1779 - val_mae: 0.1527
Epoch 42/250
4/4 [==============================] - 0s 64ms/step - loss: 0.1651 - mae: 0.1021 - val_loss: 0.0277 - val_mae: 0.0573
Epoch 43/250
4/4 [==============================] - 0s 64ms/step - loss: 0.0927 - mae: 0.0721 - val_loss: 0.0328 - val_mae: 0.0579
############### next fold => number 20 ########## 
Epoch 1/250
4/4 [============================

Epoch 13/250
4/4 [==============================] - 0s 63ms/step - loss: 0.1157 - mae: 0.0751 - val_loss: 0.0118 - val_mae: 0.0560
Epoch 14/250
4/4 [==============================] - 0s 64ms/step - loss: 0.1033 - mae: 0.0744 - val_loss: 0.0465 - val_mae: 0.0612
Epoch 15/250
4/4 [==============================] - 0s 66ms/step - loss: 0.1109 - mae: 0.0752 - val_loss: 0.0118 - val_mae: 0.0562
Epoch 16/250
4/4 [==============================] - 0s 64ms/step - loss: 0.1068 - mae: 0.0750 - val_loss: 0.0406 - val_mae: 0.0569
############### next fold => number 22 ########## 
Epoch 1/250
4/4 [==============================] - 3s 189ms/step - loss: 0.3110 - mae: 0.2113 - val_loss: 0.0529 - val_mae: 0.0735
Epoch 2/250
4/4 [==============================] - 0s 63ms/step - loss: 0.0175 - mae: 0.0463 - val_loss: 0.0127 - val_mae: 0.0557
Epoch 3/250
4/4 [==============================] - 0s 63ms/step - loss: 0.0234 - mae: 0.0522 - val_loss: 0.0108 - val_mae: 0.0500
Epoch 4/250
4/4 [=================

### Exporting and viewing results

In [43]:
df_result = read_result()
df_result = df_result.sort_values(by="mean_score", ascending = True)
df_result

,name,fold_score,mean_score,min_score,max_score,hyperparams,date
20,LinearReg,"[0.06422, 0.08158, 0.10233, 0.10397, 0.0928, 0...",0.08007,0.03786,0.11896,"{'alpha': 15, 'l1_ratio': 0.0001}",09-03 12:29:18
46,RNN,"[0.08021, 0.07414, 0.15304, 0.09484, 0.09262, ...",0.08496,0.04367,0.15304,"{'L1': 0.09621870580119067, 'L2': 0.0459439662...",09-03 13:50:07
45,RNN,"[0.25442, 0.06626, 0.10818, 0.12145, 0.09141, ...",0.09662,0.04369,0.25442,"{'L1': 0.057297844739236325, 'L2': 0.021727812...",09-03 13:16:23
21,Dummy,"[0.06752, 0.08603, 0.11788, 0.13216, 0.14218, ...",0.11057,0.05455,0.19172,NaN,09-03 12:29:22
42,RNN,"[0.14024, 0.06254, 0.10764, 0.08193, 0.24885, ...",0.13397,0.05175,0.50838,"{'L1': 0.014348080048301116, 'L2': 0.075800478...",09-03 12:58:59
10,LinearReg,"[0.13612, 0.21353, 0.30214, 0.10699, 0.19859, ...",0.16727,0.04825,0.48190,"{'alpha': 0.0146728420498667, 'l1_ratio': 0.002}",09-03 12:24:17
38,LinearReg,"[0.13771, 0.21647, 0.30717, 0.10733, 0.20077, ...",0.16939,0.04850,0.48631,"{'alpha': 0.013972992093995074, 'l1_ratio': 0....",09-03 12:29:47
25,LinearReg,"[0.13809, 0.21718, 0.30844, 0.1074, 0.20131, 0...",0.16991,0.04856,0.48734,"{'alpha': 0.013810108835731945, 'l1_ratio': 0....",09-03 12:29:33
17,LinearReg,"[0.14206, 0.22432, 0.32162, 0.10804, 0.20692, ...",0.17525,0.04917,0.49749,"{'alpha': 0.012252161010179409, 'l1_ratio': 0....",09-03 12:24:24
11,LinearReg,"[0.14258, 0.22526, 0.32338, 0.10813, 0.20769, ...",0.17597,0.04925,0.49874,"{'alpha': 0.012059999245888445, 'l1_ratio': 0....",09-03 12:24:18
